# Dynamic Programming for Global Aligment

The previous procedure is sufciciente to calculate the score of the best alignment but not the alignment itself.
We need to keep track of the decisions that define the path to reach from the beginning.



In [64]:
# Fill matrices S and T (1 for diagonal, 2 for vertical and 3 for horizontal)

def needleman_wunsch(seq1, seq2, sm, g):
    m = len(seq1)
    n = len(seq2)
    
    score = [[0]]
    trace = [[0]]
    
       
    # initialize gaps in rows
    score[0] = [g * j for j in range(0, m+1)]
    trace[0] = [3 for _ in range(0, n+1)]
    
    
    # initialize gaps in cols
    for i in range(1, m+1):
        score.append([g*i])
        trace.append([2])

    # apply the recurrence to fill the matrices
    for i in range(1, m+1):
        for j in range(1, n+1):
            s1 = score[i-1][j-1] + score_col_alignment(seq1[i-1], seq2[j-1], sm, g)
            s2 = score[i-1][j] + g
            s3 = score[i][j-1] + g
            score[i].append(max(s1,s2,s3))
            trace[i].append(max3t(s1,s2,s3))
    
    return (score, trace)

def max3t(v1,v2,v3):
    if v1 > v2:
        return 1 if v1 > v3 else 3
    else:
        return 2 if v2 > v3 else 3
            

In [65]:
# Implements a substitution matrix given an alphabet, a value for match and a value for a mismatch
def subst_matrix(alphabet, match=1, mismatch=0):
    dic = { x+y:match if x==y else mismatch for x in alphabet for y in alphabet}
    return dic

# Provides the score of a column alignment, i.e., between characters c1 and c2
# Assume a constant gap penalty g and a substitution matrix sm
def score_col_alignment(c1, c2, sm, g):
    return g if c1 == '-' or c2 == '-' else sm[c1+c2]


In [81]:
#sm = subst_matrix(['A','T','C','G'], 3, -1)
sm = read_submat_file('blosum62.mat')
 
(s,t) = needleman_wunsch('PHSWG','HGWAG', sm, -8)

for line in s:
    print(line)
print()

for line in t:
    print(line)

[0, -8, -16, -24, -32, -40]
[-8, -2, -10, -18, -25, -33]
[-16, 0, -4, -12, -20, -27]
[-24, -8, 0, -7, -11, -19]
[-32, -16, -8, 11, 3, -5]
[-40, -24, -10, 3, 11, 9]

[3, 3, 3, 3, 3, 3]
[2, 1, 3, 3, 1, 3]
[2, 1, 1, 3, 3, 1]
[2, 2, 1, 1, 1, 3]
[2, 2, 2, 1, 3, 3]
[2, 2, 1, 2, 1, 1]


In [72]:
# Recover the optimal alignment from T and A and B
# Starts bottom right corner
# Diagonal symbol in T adds symbols from A and B to the alignment
# A vertical cell in T leads to move to previous low and adds to the the alignment a symbol from A in the respective row and gap from B.
# Vice-versa if horizontal cell -> gap from A and symbol from B

DIAGONAL = 1
VERTICAL = 2
HORIZONTAL = 3

def recover_align(trace, seq1, seq2):
    res = ["",""]
    (i,j) = (len(seq1),len(seq2))
    
    while i>0 or j>0:
        if trace[i][j] is DIAGONAL:
            res[0] += seq1[i-1]
            res[1] += seq2[j-1]
            i -= 1; j -= 1
        elif trace[i][j] is HORIZONTAL:
            res[0] += '_'
            res[1] += seq2[j-1]
            j -= 1
        elif trace[i][j] is VERTICAL:
            res[0] += seq1[i-1]
            res[1] += '_'
            i -= 1
    
    return [s[::-1] for s in res]

In [68]:
# In practice for protein substitution matrix it is ieasier to read from a table or an existing file
# Assume the matrix is symmetric and first row contains the symbols in the alphabet
# Tip: use blosum62.mat

def read_submat_file(filename):
    
    with open(filename) as f:
        
        chars = f.readline().strip().replace('\t','')
    
        keys = [x for x in chars]
        matrix = []
        
        for line in f: 
            matrix.append(line.strip().split())
        
        dic = { x+y:int(matrix[yi][xi]) for xi,x in enumerate(keys) for yi,y in enumerate(keys) }
        return dic
    

In [79]:
sm = read_submat_file('blosum62.mat')

(s,t) = needleman_wunsch('PHSWG','HGWAG', sm, -3)

recover_align(t, 'PHSWG', 'HGWAG')


['PHSW_G', '_HGWAG']

## Smith-Waterman Algorithm for local alignment

In [82]:
def smith_waterman(seq1, seq2, sm, g):
    """Local alignment"""
    S = [[0]]
    T = [[0]]
    maxscore = 0
    # first row filled with zero
    for j in range(1, len(seq2)+1):
        S[0].append(0)
        T[0].append(0)
    # first column filled with zero
    for i in range(1, len(seq1)+1):
        S.append([0])
        T.append([0])
    for i in range(0, len(seq1)):
        for j in range(len(seq2)):
            s1 = S[i][j] + score_col_alignment(seq1[i], seq2[j], sm, g);
            s2 = S[i][j+1] + g
            s3 = S[i+1][j] + g
            b = max(s1, s2, s3)
            if b <= 0:
                S[i+1].append(0)
                T[i+1].append(0)
            else:
                S[i+1].append(b)
                T[i+1].append(max3t(s1, s2, s3))
                if b > maxscore:
                    maxscore = b
    return (S, T, maxscore) 

In [83]:
def max_mat(mat):
    """finds the max cell in the matrix"""
    maxval = mat[0][0]
    maxrow = 0
    maxcol = 0
    for i in range(0,len(mat)):
        for j in range(0, len(mat[i])):
            if mat[i][j] > maxval:
                maxval = mat[i][j]
                maxrow = i
                maxcol = j
    return (maxrow, maxcol)

In [84]:
def recover_align_local (S, T, seq1, seq2):
    """recover one of the optimal alignments"""
    res = ["", ""]
    """determine the cell with max score"""
    i, j = max_mat(S)
    """terminates when finds a cell with zero"""
    while T[i][j]>0:
        if T[i][j]==1:
            res[0] = seq1[i-1] + res[0]
            res[1] = seq2[j-1] + res[1]
            i -= 1
            j -= 1
        elif T[i][j] == 3:
            res[0] = "-" + res[0];
            res[1] = seq2[j-1] + res[1]
            j -= 1
        elif T[i][j] == 2:
            res[0] = seq1[i-1] + res[0]
            res[1] = "-" + res[1]
            i -= 1
    return res 

In [85]:
def test_local_alig():
    sm = read_submat_file('blosum62.mat')
    seq1 = "PHSWG"
    seq2 = "HGWAG"
    res = smith_waterman(seq1, seq2, sm, -8)
    S = res[0]
    T = res[1]
    print("Score of optimal alignment:", res[2])
    #print_mat(S)
    #print_mat(T)
    print(S)
    print(T)
    alinL = recover_align_local(S, T, seq1, seq2)
    print(alinL[0])
    print(alinL[1]) 

In [86]:
test_local_alig()

Score of optimal alignment: 19
[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 8, 0, 0, 0, 0], [0, 0, 8, 0, 1, 0], [0, 0, 0, 19, 11, 3], [0, 0, 6, 11, 19, 17]]
[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0], [0, 0, 0, 1, 3, 3], [0, 0, 1, 2, 1, 1]]
HSW
HGW


In [87]:
# Write a function that finds the most similar sequence to a query sequence.
# The function should use the local alignment algorithm developed previously
# It receives as input:
#    - Query sequence (query)
#    - List of sequences (list_of_seqs)
#    - Substitution matrix (sm)
#    - Gap penalty (g)

def find_most_similar(query, list_of_seqs, sm, g):
    res = max([(smith_waterman(query, seq, sm, g), seq) for seq in list_of_seqs], key=lambda x: x[0][2])
    
    alignment = recover_align_local(res[0][0], res[0][1], query, res[1])
    score = res[0][2]
    return (alignment, score)
# Pode sair em exame
        
    

In [88]:
sm = read_submat_file('blosum62.mat')
find_most_similar('ACTG', ['ATGC', 'ATTT', 'TACT', 'ATTGT'], sm, g=1)

(['ACTG', 'ACT-'], 19)

# Exercises

Calculate the identify funciton between two sequences. It should rueturn a value of 0 to 1, where 1 is `seq1 == seq2`

- Calculate the edit distance between two sequences. As we have seen before, edit distance is the minimum of 

# Trabalho 2

`global_align_multiple_solutions` e `recover_global_align_mutliple_solutions`



In [ ]:
def global_align_multiple_solutions()

## BLAST

- Basic Local Alignment Search Tool
- Set of programs
- For nucleotides and protein sequences
- Matches sequences against sequence databases



In [56]:
def max_list(*argv):
    m = max(argv)
    return list(filter(lambda x: x == m, argv))

In [57]:
max_list(1,2,2,3)

[3]

In [59]:
def max_3t_list(*argv):
    m = max(argv)
    res = []
    for i,v in enumerate(argv):
        if(v == m):
            res.append(i)
    return res

In [62]:
max_3t_list(3)

[0]

In [130]:
mat = [[random.randint(0,10) for j in range(0,5)] for i in range(0,5)]

for line in mat:
    print(line)

max_value = max([max(line) for line in mat])
print(max_value)

[(i,j) for i,line in enumerate(mat) for j,v in enumerate(line) if v is max_value]

[6, 4, 7, 2, 4]
[5, 5, 6, 3, 8]
[4, 1, 1, 6, 5]
[5, 10, 5, 10, 7]
[10, 6, 9, 2, 3]
10


[(3, 1), (3, 3), (4, 0)]

In [112]:
random.randint(0,10)

2

In [6]:
asd = [['CNNITDVDP-CVFCSSPTRNQRLVCVVEEPTNIAAVEKTRG-YNGVYHVLHGTLSPLHGVGPEHLRTVN-LLARVERGEVD--ELILATSPTVEGEATANYLADLLRPLRVRLTRIATGVPAGSDIEYVDEVTMTRALEGRREL', 'CNNLSEA-PLCAVCRSEHRDRSILCVVESPADLRAIEDT-GVFVGEFFVLMGHLSPLDGIGPEALH-IDRLSARL--GETDLQEVIFATNPTLEGEATAQFLAGLV-PDGVTISRIARGVPVGGELEYVDRSTLGRALHGRR-L'], ['CNNITDVDP-CVFCSSPTRNQRLVCVVEEPTNIAAVEKTRG-YNGVYHVLHGTLSPLHGVGPEHLRTVNLL-ARVERGEVD--ELILATSPTVEGEATANYLADLLRPLRVRLTRIATGVPAGSDIEYVDEVTMTRALEGRREL', 'CNNLSEA-PLCAVCRSEHRDRSILCVVESPADLRAIEDT-GVFVGEFFVLMGHLSPLDGIGPEALH-IDRLSARL--GETDLQEVIFATNPTLEGEATAQFLAGLV-PDGVTISRIARGVPVGGELEYVDRSTLGRALHGRR-L'], ['CNNITDVDP-CVFCSSPTRNQRLVCVVEEPTNIAAVEKTRG-YNGVYHVLHGTLSPLHGVGPEHLRTVN-LLARVERGEVD--ELILATSPTVEGEATANYLADLLRPLRVRLTRIATGVPAGSDIEYVDEVTMTRALEGRREL', 'CNNLSEA-PLCAVCRSEHRDRSILCVVESPADLRAIEDT-GVFVGEFFVLMGHLSPLDGIGPEALH-IDRLSARL--GETDLQEVIFATNPTLEGEATAQFLAGLV-PDGVTISRIARGVPVGGELEYVDRSTLGRALHGRRLL'], ['CNNITDVDP-CVFCSSPTRNQRLVCVVEEPTNIAAVEKTRG-YNGVYHVLHGTLSPLHGVGPEHLRTVNLL-ARVERGEVD--ELILATSPTVEGEATANYLADLLRPLRVRLTRIATGVPAGSDIEYVDEVTMTRALEGRREL', 'CNNLSEA-PLCAVCRSEHRDRSILCVVESPADLRAIEDT-GVFVGEFFVLMGHLSPLDGIGPEALH-IDRLSARL--GETDLQEVIFATNPTLEGEATAQFLAGLV-PDGVTISRIARGVPVGGELEYVDRSTLGRALHGRRLL']]

In [5]:
set(asd)

TypeError: unhashable type: 'list'

In [1]:
[['CNNITDVDP_CVFCSSPTRNQRLVCVVEEPTNIAAVEKTRG_YNGVYHVLHGTLSPLHGVGPEHLRTVNLL_ARVERGEVD__ELILATSPTVEGEATANYLADLLRPLRVRLTRIATGVPAGSDIEYVDEVTMTRALEGRREL', 'CNNLSEA_PLCAVCRSEHRDRSILCVVESPADLRAIEDT_GVFVGEFFVLMGHLSPLDGIGPEALH_IDRLSARL__GETDLQEVIFATNPTLEGEATAQFLAGLV_PDGVTISRIARGVPVGGELEYVDRSTLGRALHGRR_L'], ['CNNITDVDP_CVFCSSPTRNQRLVCVVEEPTNIAAVEKTRG_YNGVYHVLHGTLSPLHGVGPEHLRTVN_LLARVERGEVD__ELILATSPTVEGEATANYLADLLRPLRVRLTRIATGVPAGSDIEYVDEVTMTRALEGRREL', 'CNNLSEA_PLCAVCRSEHRDRSILCVVESPADLRAIEDT_GVFVGEFFVLMGHLSPLDGIGPEALH_IDRLSARL__GETDLQEVIFATNPTLEGEATAQFLAGLV_PDGVTISRIARGVPVGGELEYVDRSTLGRALHGRR_L']]

[['CNNITDVDP_CVFCSSPTRNQRLVCVVEEPTNIAAVEKTRG_YNGVYHVLHGTLSPLHGVGPEHLRTVNLL_ARVERGEVD__ELILATSPTVEGEATANYLADLLRPLRVRLTRIATGVPAGSDIEYVDEVTMTRALEGRREL',
  'CNNLSEA_PLCAVCRSEHRDRSILCVVESPADLRAIEDT_GVFVGEFFVLMGHLSPLDGIGPEALH_IDRLSARL__GETDLQEVIFATNPTLEGEATAQFLAGLV_PDGVTISRIARGVPVGGELEYVDRSTLGRALHGRR_L'],
 ['CNNITDVDP_CVFCSSPTRNQRLVCVVEEPTNIAAVEKTRG_YNGVYHVLHGTLSPLHGVGPEHLRTVN_LLARVERGEVD__ELILATSPTVEGEATANYLADLLRPLRVRLTRIATGVPAGSDIEYVDEVTMTRALEGRREL',
  'CNNLSEA_PLCAVCRSEHRDRSILCVVESPADLRAIEDT_GVFVGEFFVLMGHLSPLDGIGPEALH_IDRLSARL__GETDLQEVIFATNPTLEGEATAQFLAGLV_PDGVTISRIARGVPVGGELEYVDRSTLGRALHGRR_L']]

In [2]:
seq1 = 'CNNITDVDP-CVFCSSPTRNQRLVCVVEEPTNIAAVEKTRG-YNGVYHVLHGTLSPLHGVGPEHLRTVN-LLARVERGEVD--ELILATSPTVEGEATANYLADLLRPLRVRLTRIATGVPAGSDIEYVDEVTMTRALEGRREL'
seq2 = 'CNNITDVDP-CVFCSSPTRNQRLVCVVEEPTNIAAVEKTRG-YNGVYHVLHGTLSPLHGVGPEHLRTVN-LLARVERGEVD--ELILATSPTVEGEATANYLADLLRPLRVRLTRIATGVPAGSDIEYVDEVTMTRALEGRREL'

In [3]:
seq1 == seq2

True

In [4]:
seq3 = 'CNNLSEA-PLCAVCRSEHRDRSILCVVESPADLRAIEDT-GVFVGEFFVLMGHLSPLDGIGPEALH-IDRLSARL--GETDLQEVIFATNPTLEGEATAQFLAGLV-PDGVTISRIARGVPVGGELEYVDRSTLGRALHGRR-L'
seq4 = 'CNNLSEA-PLCAVCRSEHRDRSILCVVESPADLRAIEDT-GVFVGEFFVLMGHLSPLDGIGPEALH-IDRLSARL--GETDLQEVIFATNPTLEGEATAQFLAGLV-PDGVTISRIARGVPVGGELEYVDRSTLGRALHGRRLL'

In [15]:
print(asd[0][0] == asd[2][0])
print(asd[0][1] == asd[2][1])
print(asd[1][0] == asd[3][0])
print(asd[1][1] == asd[3][1])

True
False
True
False
